In [27]:
import numpy as np
import torch 
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
#from torchsummary import summary
import matplotlib.pyplot as plt
import os
import time
import copy
from datetime import datetime
from collections import OrderedDict
import json
from torch.utils.tensorboard import SummaryWriter
import random
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

In [28]:
data_dir = 'train_images'

time_run = datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = 'results/' + time_run
os.makedirs(save_dir, exist_ok=True)

transform_mean = transform_std = np.array([None, None, None])

In [29]:
df = pd.read_csv("train_labels.csv", names=["name", "label"], header=None)
df["image_path"] = df["name"].apply(lambda x: os.path.join(data_dir, x))
df = df[["image_path", "label"]]
df.head()

,image_path,label
0,train_images\train_0001.png,2012
1,train_images\train_0002.png,2003
2,train_images\train_0003.png,1994
3,train_images\train_0004.png,2014
4,train_images\train_0005.png,2003


In [30]:
df["image_path"][0]

'train_images\\train_0001.png'

In [31]:
n_epoch = 40
batch_size = 16
image_size = 256
input_size = 224
n_classes = 40

In [32]:
if transform_mean.any() == None:
    transform_mean = np.array([0.4914, 0.4822, 0.4465])
if transform_std.any() == None:
    transform_std = np.array([0.2470, 0.2435, 0.2616])


class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, df, data_dir, augment=True):
        
        if augment:
            transform = transforms.Compose([
                transforms.RandomCrop(128, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=transform_mean, std=transform_std)
            ])
        else:
            transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=transform_mean, std=transform_std)
            ])

        self.labels = df["label"]
        self.data_dir = data_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = df["image_path"][idx]
        image = Image.open(img_name).convert("RGB")   
        image = self.transform(image)
        
        label = df["label"][idx] - 1979
        sample = {'image': image, 'label': label}
        return sample


train_df, val_df = train_test_split(
                    df, test_size=0.2, random_state=42, shuffle=True, stratify=df["label"]
                    )    

train_dataset = ImageDataset(train_df, data_dir, augment=True)
val_dataset = ImageDataset(val_df, data_dir, augment=False)

train_df.to_csv("results/list_train.csv", header=None, index=None)
val_df.to_csv("results/list_val.csv", header=None, index=None)



train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True, 
                                           num_worers=0,
#                                           pin_memory=True,
                                           drop_last=True
                                          )

val_loader = torch.utils.data.DataLoader(val_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False, 
                                           num_workers=0,
#                                           pin_memory=True,
                                           drop_last=False
                                         )

train_dataset_size = len(train_dataset)
val_dataset_size = len(val_dataset)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

In [53]:
## Model

class Resnet152(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        resnet152 = models.resnet152(pretrained=True)
        self.resnet = torch.nn.Sequential(*(list(resnet152.children())[:-1]))
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(2048, n_classes)
        
        
    def set_requires_grad(self, requires_grad=False):
        for block in self.resnet:
            for p in block.parameters():
                p.requires_grad = requires_grad

    def forward(self, x):
        x = self.resnet(x)
        x = F.adaptive_max_pool2d(x, 1)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        out = F.softmax(self.fc(x))
        return out


In [54]:
#base_chan = 32
#n_channels = [base_chan, base_chan * 2, base_chan * 4]

#model = Network()
#model = Shakeshake(n_channels, n_classes)
model = Resnet152(n_classes)
model = model.to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4, nesterov=True)

# Decay LR by a factor of 0.1 every 7 epochs
#scheduler = lr_scheduler.StepLR(optimizer, step_size=9, gamma=0.33)

T_max = n_epoch * train_dataset_size
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1)

global_step = 0

In [55]:
## auxilliary functions

def get_config():
    config = OrderedDict({'name':time_run,
                          'nepoch':n_epoch,
#                          'base_chan':base_chan,
                          'loss_func':str(loss_func),
                          'optimizer':str(optimizer),
                          'scheduler':str(scheduler.__dict__),
                          'model':str(model),
                          'train_loader':str(train_loader.__dict__), 
                          'val_loader':str(val_loader.__dict__)})
    return config

In [56]:
def train(epoch, model, loss_func, train_loader, optimizer, scheduler, writer):
    global global_step
    
    since = time.time()

    model.train()
    
    running_loss = 0.0            
    running_correct = 0

    for i, data in enumerate(train_loader):
        global_step +=1
        inputs, labels = data["image"], data["label"]

#        if i == 0:
#            image = torchvision.utils.make_grid(
#                inputs, normalize=True, scale_each=True)
#            writer.add_image('Train/Image', image, epoch)
        
        scheduler.step()        
        
        writer.add_scalar('Train/LearningRate',
                            scheduler.get_lr()[0], global_step)
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = loss_func(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_correct += np.double(torch.sum(predicted == labels.data))
        
        writer.add_scalar('Train/RunningLoss', running_loss , global_step)
        writer.add_scalar('Train/RunningAccuracy', running_correct , global_step)
        
    train_loss = running_loss / train_dataset_size
    train_acc = running_correct / train_dataset_size

    elapsed = time.time() - since
    
    writer.add_scalar('Train/Loss', train_loss, epoch)
    writer.add_scalar('Train/Accuracy', train_acc, epoch)
    writer.add_scalar('Train/Time', elapsed, epoch)

    train_log = OrderedDict({
    'epoch':
    epoch,
    'train':
    OrderedDict({
        'loss': train_loss,
        'accuracy': train_acc,
        'time': elapsed,
        }),
    })
      
    print('Train Loss:{:.6f} Accuracy:{:.4f} \nTime:{:.1f}s'.format(train_loss, train_acc, elapsed))        
        
    return train_log, train_acc

def val(epoch, model, loss_func, val_loader, writer):
    global global_step
    since = time.time()
    
    model.eval()
    
    running_loss = 0.0            
    running_correct = 0
    
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            global_step +=1
            inputs, labels = data["image"], data["label"]
            
#            if i == 0:
#                image = torchvision.utils.make_grid(
#                    inputs, normalize=True, scale_each=True)
#                writer.add_image('Test/Image', image, epoch)
            
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)

            running_loss += loss.item()
            running_correct += np.double(torch.sum(predicted == labels.data))
    val_loss = running_loss / val_dataset_size
    val_acc = running_correct / val_dataset_size

    elapsed = time.time() - since

    if epoch > 0:
        writer.add_scalar('Val/Loss', val_loss, epoch)
    writer.add_scalar('Val/Accuracy', val_acc, epoch)
    writer.add_scalar('Val/Time', elapsed, epoch)    
    
    val_log = OrderedDict({
    'epoch':
    epoch,
    'val':
    OrderedDict({
        'loss': val_loss,
        'accuracy':val_acc,
        'time': elapsed,
        }),
    })
    
    print('Val Loss:{:.6f} Accuracy:{:.4f} \nTime:{:.1f}s'.format(val_loss, val_acc, elapsed))  
    
    return val_log, val_acc



def main():
    print(time_run)
    
    config = get_config()
    with open(save_dir+'/{}_config.json'.format(time_run), 'w') as fout:
        json.dump(config, fout, indent=2)
        
#    summary(model, (3, 32, 32))
#    print('no. epoch:', nepoch, "batch size:", BATCH_SIZE, "optimizer:", optimizer)
    train_since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

######################## This part function not verified ###################    
    # set random seed
    seed = 17 # arbitrary
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
###########################################################################    
    writer = SummaryWriter(save_dir)
    
    # run test before start training? whats that??????????
    val(0, model, loss_func, val_loader, writer)
    
    epoch_logs = []
    train_accs = []
    val_accs = []
    
    for epoch in range(n_epoch):
        print('Epoch {}/{}'.format(epoch, n_epoch - 1))
        print('-' * 10)
        
        train_log, train_acc = train(epoch, model, loss_func, train_loader, optimizer, scheduler, writer)
        val_log, val_acc = val(epoch, model, loss_func, val_loader, writer)
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        

        print()
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        epoch_log = train_log.copy()
        epoch_log.update(val_log)
        epoch_logs.append(epoch_log)
        with open(save_dir+'/{}_log.json'.format(time_run), 'w') as fout:
            json.dump(epoch_logs, fout, indent=2)
        
        if epoch % 50 == 0 and epoch !=0:
            torch.save(model.state_dict(), save_dir+'/{}_ep{}_acc{:.4f}.pth'.format(time_run, epoch, val_acc))
    
    accuracies = {'train_accuracy':np.array(train_accs), 'val_accuracy':np.array(val_accs)}
    np.save(save_dir+'/{}_accs.npy'.format(time_run), accuracies)
    
    train_elapsed = time.time() - train_since
    print('Training complete in {:.0f}m {:.0f}s'.format(train_elapsed // 60, train_elapsed % 60))
    print('Best val Acc: {:.4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), save_dir+'/{}_bestacc{:.4f}.pth'.format(time_run, best_acc))

In [ ]:
if __name__ == '__main__':
    main()

20191228_233933


C:\Users\silas\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Val Loss:0.231539 Accuracy:0.0321 
Time:10.3s
Epoch 0/39
----------
Train Loss:0.228548 Accuracy:0.0673 
Time:94.9s
Val Loss:0.227789 Accuracy:0.0972 
Time:9.9s

Epoch 1/39
----------
Train Loss:0.227093 Accuracy:0.0909 
Time:88.8s
Val Loss:0.226531 Accuracy:0.1226 
Time:9.8s

Epoch 2/39
----------
Train Loss:0.226443 Accuracy:0.1038 
Time:90.5s
Val Loss:0.226375 Accuracy:0.1233 
Time:9.8s

Epoch 3/39
----------
Train Loss:0.227402 Accuracy:0.0886 
Time:88.8s
Val Loss:0.228577 Accuracy:0.0882 
Time:9.9s

Epoch 4/39
----------
Train Loss:0.227675 Accuracy:0.0843 
Time:88.0s
Val Loss:0.227388 Accuracy:0.1061 
Time:9.8s

Epoch 5/39
----------
Train Loss:0.227587 Accuracy:0.0847 
Time:87.8s
Val Loss:0.227681 Accuracy:0.1001 
Time:9.9s

Epoch 6/39
----------
Train Loss:0.227796 Accuracy:0.0815 
Time:87.7s
Val Loss:0.228991 Accuracy:0.0792 
Time:10.0s

Epoch 7/39
----------
